In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

# Run in terminal or command prompt
!python3 -m spacy download en


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/preetgandhi95/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!

    Linking successful
    /Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/en_core_web_sm
    -->
    /Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [5]:
lyr=pd.read_csv('merged_eng_lyrics_genre_explicit.csv')

In [6]:
def repl(e):
    return [re.sub('\n', ' ', sent) for sent in e]

In [7]:
blues=repl(list(lyr[lyr['Blues']==1]['lyrics']))
cg=repl(list(lyr[lyr['Christian/Gospel']==1]['lyrics']))
country=repl(list(lyr[lyr['Country']==1]['lyrics']))
de=repl(list(lyr[lyr['Dance/Electro']==1]['lyrics']))
disco=repl(list(lyr[lyr['Disco']==1]['lyrics']))
dh=repl(list(lyr[lyr['Dutch-House']==1]['lyrics']))
folk=repl(list(lyr[lyr['Folk']==1]['lyrics']))
hiphop=repl(list(lyr[lyr['Hip-Hop']==1]['lyrics']))
indie=repl(list(lyr[lyr['Indie']==1]['lyrics']))
jazz=repl(list(lyr[lyr['Jazz']==1]['lyrics'] ) )
latin=repl(list(lyr[lyr['Latin']==1]['lyrics']))
metal=repl(list(lyr[lyr['Metal']==1]['lyrics']))
other=repl(list(lyr[lyr['Other']==1]['lyrics']))
pop=repl(list(lyr[lyr['Pop']==1]['lyrics']))
rb=repl(list(lyr[lyr['R&B']==1]['lyrics']))
reggae=repl(list(lyr[lyr['Reggae']==1]['lyrics']))
ra=repl(list(lyr[lyr['Rock/Alt']==1]['lyrics']))
rap=repl(list(lyr[lyr['Rock/Alt/Pop']==1]['lyrics']))
missing=repl(list(lyr[lyr['missing']==1]['lyrics']))
explicit=repl(list(lyr[lyr['Explicit']==1]['lyrics']))



In [8]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

blues_words = list(sent_to_words(blues))
cg_words = list(sent_to_words(cg))
country_words = list(sent_to_words(country))
de_words = list(sent_to_words(de))
disco_words = list(sent_to_words(disco))
dh_words = list(sent_to_words(dh))
folk_words = list(sent_to_words(folk))
hiphop_words = list(sent_to_words(hiphop))
jazz_words = list(sent_to_words(jazz))
latin_words = list(sent_to_words(latin))
metal_words = list(sent_to_words(metal))
other_words = list(sent_to_words(other))
pop_words = list(sent_to_words(pop))
rb_words = list(sent_to_words(rb))
reggae_words = list(sent_to_words(reggae))
ra_words = list(sent_to_words(ra))
rap_words = list(sent_to_words(rap))
missing_words = list(sent_to_words(missing))
explicit_words = list(sent_to_words(explicit))

In [9]:
def build(wrd):
# Build the bigram and trigram models
    bigram = gensim.models.Phrases(wrd, min_count=1, threshold=1) # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[wrd], threshold=1)  

# Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return bigram_mod,trigram_mod

blues_b,blues_t=build(blues_words)
cg_b,cg_t=build(cg_words)
country_b,country_t=build(country_words)
de_b,de_t=build(de_words)
disco_b,disco_t=build(disco_words)
dh_b,dh_t=build(dh_words)
folk_b,folk_t=build(folk_words)
hiphop_b,hiphop_t=build(hiphop_words)
jazz_b,jazz_t=build(jazz_words)
latin_b,latin_t=build(latin_words)
metal_b,metal_t=build(metal_words)
other_b,other_t=build(other_words)
pop_b,pop_t=build(pop_words)
rb_b,rb_t=build(rb_words)
reggae_b,reggae_t=build(reggae_words)
ra_b,ra_t=build(ra_words)
rap_b,rap_t=build(rap_words)
missing_b,missing_t=build(missing_words)
explicit_b,explicit_t=build(explicit_words)    

In [17]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts,bi):
    return [bi[doc] for doc in texts]

def make_trigrams(texts,tri,bi):
    return [tri[bi[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        nlp = spacy.load('en', disable=['parser', 'ner'])
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [18]:
def lem(data_words,bi,tri):# Remove Stop Words
    data_words_nostops = remove_stopwords(data_words)

    # Form Bigrams
    data_words_bigrams = make_bigrams(data_words_nostops,bi)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en', disable=['parser', 'ner'])

    # Do lemmatization keeping only noun, adj, vb, adv
    return lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



In [ ]:
# blues_lem=lem(blues_words)
# cg_lem=lem(cg_words)
# country_lem=lem(country_words)
# de_lem=lem(de_words)
# disco_lem=lem(disco_words)
# dh_lem=lem(dh_words)
# folk_lem=lem(folk_words)
hiphop_lem=lem(hiphop_words,hiphop_b,hiphop_t)
# jazz_lem=lem(jazz_words)
# latin_lem=lem(latin_words)
# metal_lem=lem(metal_words)
# other_lem=lem(other_words)
# pop_lem=lem(pop_words)
# rb_lem=lem(rb_words)
# reggae_lem =lem(reggae_words)
# ra_lem=lem(ra_words)
# rap_lem=lem(rap_words)
# missing_lem=lem(missing_words)
# explicit_lem=lem(explicit_words)

In [78]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]



In [81]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('because', 1),
  ('broken_heart', 1),
  ('care', 1),
  ('clown', 7),
  ('come', 1),
  ('cry', 1),
  ('die', 1),
  ('dream', 4),
  ('easy', 1),
  ('everybody', 3),
  ('feeling', 1),
  ('guess', 1),
  ('guy', 1),
  ('joke', 2),
  ('know', 5),
  ('laugh', 4),
  ('little_closer', 1),
  ('look', 2),
  ('love', 7),
  ('part', 1),
  ('party', 1),
  ('play', 1),
  ('run', 1),
  ('say', 2),
  ('say_love', 1),
  ('see', 2),
  ('side', 1),
  ('smile', 1),
  ('start', 4),
  ('tell', 1),
  ('thing', 1),
  ('want', 1),
  ('wonder', 1),
  ('would', 1)]]

In [82]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [83]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.037*"wanna" + 0.031*"duck" + 0.024*"saturday" + 0.016*"mama" + '
  '0.015*"dealer" + 0.014*"wall" + 0.014*"wig" + 0.014*"trudi" + 0.012*"get" + '
  '0.010*"motor"'),
 (1,
  '0.017*"look" + 0.015*"tonight" + 0.011*"moon" + 0.010*"music" + '
  '0.010*"gonna_make" + 0.009*"catch" + 0.009*"star" + 0.009*"life" + '
  '0.008*"doo" + 0.007*"breathe"'),
 (2,
  '0.043*"want" + 0.032*"tired" + 0.026*"baby" + 0.016*"keep" + 0.015*"get" + '
  '0.014*"mouth" + 0.012*"despair" + 0.010*"money" + 0.009*"fine" + '
  '0.009*"sittin"'),
 (3,
  '0.036*"know" + 0.033*"go" + 0.026*"love" + 0.016*"come" + 0.016*"day" + '
  '0.015*"get" + 0.014*"baby" + 0.014*"heart" + 0.012*"see" + 0.012*"girl"'),
 (4,
  '0.008*"day" + 0.008*"water" + 0.007*"say" + 0.007*"glenarvan" + '
  '0.006*"boat" + 0.006*"make" + 0.006*"shore" + 0.005*"birthday" + 0.005*"go" '
  '+ 0.004*"sea"'),
 (5,
  '0.055*"get" + 0.015*"shit" + 0.010*"try" + 0.009*"bump" + 0.009*"go" + '
  '0.009*"say" + 0.009*"really" + 0.008*"business"

In [84]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.141460404241398

Coherence Score:  0.3827779398472426


In [85]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/preetgandhi95/miniconda3/envs/nlpclass/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.252988 -0.046616       1        1  22.175173
17     0.166119 -0.115339       2        1  18.712214
3      0.272531  0.026137       3        1  10.483575
9      0.126069 -0.011078       4        1   9.727961
4      0.058113 -0.172102       5        1   7.466518
5      0.137403  0.143712       6        1   6.668943
15    -0.000367 -0.150662       7        1   4.297186
7      0.017578  0.187927       8        1   3.634248
16     0.094911  0.109205       9        1   2.429289
6     -0.172387  0.020699      10        1   2.323248
1     -0.055739 -0.044517      11        1   2.242406
14    -0.045831  0.081817      12        1   2.102705
8     -0.080710 -0.009158      13        1   1.489714
0     -0.100956  0.022857      14        1   1.436406
2     -0.062948  0.042453      15        1   1.218755
18    -0.097116 -0.005670      16        1   1.088529
19    -0.125885 -0.036261      17        1   0.987380
10    -0.152232 -0.021771      18        1   0.678742
13    -0.109004 -0.018085      19        1   0.524899
12    -0.122535 -0.003549      20        1   0.312104, topic_info=     Category        Freq           Term       Total  loglift  logprob
term                                                                  
18    Default  337.000000           love  337.000000  30.0000  30.0000
6797  Default  351.000000          shrek  351.000000  29.0000  29.0000
6493  Default  273.000000         donkey  273.000000  28.0000  28.0000
48    Default  545.000000            get  545.000000  27.0000  27.0000
153   Default  207.000000            man  207.000000  26.0000  26.0000
14    Default  512.000000           know  512.000000  25.0000  25.0000
31    Default  203.000000           want  203.000000  24.0000  24.0000
29    Default  214.000000           tell  214.000000  23.0000  23.0000
6536  Default  169.000000          fiona  169.000000  22.0000  22.0000
49    Default  523.000000             go  523.000000  21.0000  21.0000
76    Default  129.000000           baby  129.000000  20.0000  20.0000
105   Default  182.000000           good  182.000000  19.0000  19.0000
914   Default  113.000000           head  113.000000  18.0000  18.0000
17    Default  240.000000           look  240.000000  17.0000  17.0000
862   Default  225.000000            way  225.000000  16.0000  16.0000
6206  Default   56.000000            goo   56.000000  15.0000  15.0000
142   Default  132.000000           girl  132.000000  14.0000  14.0000
378   Default  134.000000           back  134.000000  13.0000  13.0000
127   Default  260.000000           time  260.000000  12.0000  12.0000
595   Default  117.000000           keep  117.000000  11.0000  11.0000
23    Default  473.000000            say  473.000000  10.0000  10.0000
1636  Default   82.000000          water   82.000000   9.0000   9.0000
133   Default   97.000000           away   97.000000   8.0000   8.0000
654   Default   70.000000           name   70.000000   7.0000   7.0000
50    Default  127.000000          heart  127.000000   6.0000   6.0000
25    Default  343.000000            see  343.000000   5.0000   5.0000
30    Default  162.000000          thing  162.000000   4.0000   4.0000
130   Default  135.000000           walk  135.000000   3.0000   3.0000
838   Default  134.000000            try  134.000000   2.0000   2.0000
903   Default   48.000000           move   48.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
7584  Topic20    0.531260    persecution    1.218620   4.9394  -5.8722
1157  Topic20    0.457499      take_good    1.144853   4.8523  -6.0216
916   Topic20    0.751377        oh_hold    2.092206   4.7455  -5.5255
915   Topic20    0.383737       let_burn    1.071085   4.7431  -6.1975
1158  Topic20    0.383724     think_love    1.071107   4.7431  -6.1975
6855  Topic20    1.236918   talking_well    3.748211   4.6609  -5.0270
